In [242]:
import os
import pandas as pd

In [243]:
#create filepath
school_filepath = os.path.join("raw_data","schools_complete.csv")
student_filepath = os.path.join("raw_data","students_complete.csv")

In [244]:
#read csv to data-frames
school_df = pd.read_csv(school_filepath)
student_df = pd.read_csv(student_filepath)

In [245]:
school_df.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [246]:
student_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [247]:
#use groupby on student_df to get the avg reading/math score for each school
stu_df1 = student_df.groupby("school")["reading_score","math_score"].mean()
stu_df1

,reading_score,math_score
school,,
Bailey High School,81.033963,77.048432
Cabrera High School,83.975780,83.061895
Figueroa High School,81.158020,76.711767
Ford High School,80.746258,77.102592
Griffin High School,83.816757,83.351499
Hernandez High School,80.934412,77.289752
Holden High School,83.814988,83.803279
Huang High School,81.182722,76.629414
Johnson High School,80.966394,77.072464


In [248]:
#use column filter on student_df to find the % of students passing math & reading(assuume 70 is pass)
math_pass_df = student_df.loc[student_df["math_score"] > 60, :]

reading_pass_df = student_df.loc[student_df["reading_score"] > 60, :]

In [249]:
#use groupby on each of the df to get the total passing students per school
math_pass_count = pd.DataFrame(math_pass_df.groupby("school")["name"].count())
#rename "name" column 
math_pass_count = math_pass_count.rename(columns = {"name":"total math pass"})

reading_pass_count = pd.DataFrame(reading_pass_df.groupby("school")["name"].count())
#rename "name" column 
reading_pass_count = reading_pass_count.rename(columns = {"name":"total reading pass"})


In [250]:
#Use df.join() to join these two df to stu_df1
stu_df2 = stu_df1.join(math_pass_count, how = "outer")
stu_df3 = stu_df2.join(reading_pass_count, how = "outer") 
stu_df3.head()

,reading_score,math_score,total math pass,total reading pass
school,,,,
Bailey High School,81.033963,77.048432,4351,4976
Cabrera High School,83.975780,83.061895,1858,1858
Figueroa High School,81.158020,76.711767,2549,2949
Ford High School,80.746258,77.102592,2389,2739
Griffin High School,83.816757,83.351499,1468,1468


In [251]:
#make the index of stu_df4(school) into a column
stu_df3.reset_index(level=0, inplace=True)
stu_df3.head()

,school,reading_score,math_score,total math pass,total reading pass
0,Bailey High School,81.033963,77.048432,4351,4976
1,Cabrera High School,83.975780,83.061895,1858,1858
2,Figueroa High School,81.158020,76.711767,2549,2949
3,Ford High School,80.746258,77.102592,2389,2739
4,Griffin High School,83.816757,83.351499,1468,1468


In [252]:
#rename "name" column in school_df to "school" to facilitate merging
school_df = school_df.rename(columns = {"name":"school"})
school_df.head()

,School ID,school,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [253]:
#merge school_df & stu_df4
merge_df = pd.merge(school_df, stu_df3, on = "school", how = "outer")
merge_df.head()

,School ID,school,type,size,budget,reading_score,math_score,total math pass,total reading pass
0,0,Huang High School,District,2917,1910635,81.182722,76.629414,2533,2917
1,1,Figueroa High School,District,2949,1884411,81.158020,76.711767,2549,2949
2,2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,1761,1761
3,3,Hernandez High School,District,4635,3022020,80.934412,77.289752,4007,4635
4,4,Griffin High School,Charter,1468,917500,83.816757,83.351499,1468,1468


In [254]:
#calculate the pass % per school for reading & math
merge_df["% passing math"] = (merge_df["total math pass"]/merge_df["size"])*100
merge_df["% passing reading"] = (merge_df["total reading pass"]/merge_df["size"])*100
merge_df.head()

,School ID,school,type,size,budget,reading_score,math_score,total math pass,total reading pass,% passing math,% passing reading
0,0,Huang High School,District,2917,1910635,81.182722,76.629414,2533,2917,86.835790,100.0
1,1,Figueroa High School,District,2949,1884411,81.158020,76.711767,2549,2949,86.436080,100.0
2,2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,1761,1761,100.000000,100.0
3,3,Hernandez High School,District,4635,3022020,80.934412,77.289752,4007,4635,86.450917,100.0
4,4,Griffin High School,Charter,1468,917500,83.816757,83.351499,1468,1468,100.000000,100.0


In [255]:
#create a df with only information about district schools
district_df = merge_df.loc[merge_df["type"] == "District", : ]

In [256]:
#get data from district_df 
total_district_schools = district_df["school"].count()
total_district_students = district_df["size"].sum()
total_district_budget = district_df["budget"].sum()
avg_math_score = district_df["math_score"].mean()
avg_reading_score = district_df["reading_score"].mean()
prcnt_passing_math = district_df["% passing math"].mean()
prcnt_passing_reading = district_df["% passing reading"].mean()
overall_passing_rate = (prcnt_passing_math + prcnt_passing_reading)/2

In [257]:
#create a data frame from the district data
district_summary = pd.DataFrame({"Overall Passing Rate" : [overall_passing_rate],
                                 "% Passing Reading" : [prcnt_passing_reading],
                                 "% Passing Math": [prcnt_passing_math],
                                 "Average Reading Score" : [avg_reading_score],
                                 "Average Math Score" : [avg_math_score],
                                 "Total Budget" :[total_district_budget],
                                 "Total Students" : [total_district_students],
                                 "Total Schools" : [total_district_schools],
                                })
district_summary

,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Overall Passing Rate,Total Budget,Total Schools,Total Students
0,86.790742,100.0,76.956733,80.966636,93.395371,17347923,7,26976


In [258]:
#arrange items in the correct order
district_summary.loc[:,["Total Schools","Total Students","Total Budget","Average Math Score","Average Reading Score",\
                       "% Passing Math","% Passing Reading","Overall Passing Rate"]]

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,7,26976,17347923,76.956733,80.966636,86.790742,100.0,93.395371


In [259]:
merge_df.head()

,School ID,school,type,size,budget,reading_score,math_score,total math pass,total reading pass,% passing math,% passing reading
0,0,Huang High School,District,2917,1910635,81.182722,76.629414,2533,2917,86.835790,100.0
1,1,Figueroa High School,District,2949,1884411,81.158020,76.711767,2549,2949,86.436080,100.0
2,2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,1761,1761,100.000000,100.0
3,3,Hernandez High School,District,4635,3022020,80.934412,77.289752,4007,4635,86.450917,100.0
4,4,Griffin High School,Charter,1468,917500,83.816757,83.351499,1468,1468,100.000000,100.0


In [260]:
#Copy merge_df to school_sumry_df, set school name as index & delete unwanted columsn
school_sumry_df = merge_df.copy()
school_sumry_df.set_index("school", inplace=True)
del school_sumry_df["School ID"]
school_sumry_df = school_sumry_df.rename(columns = {"size":"Total students"})
school_sumry_df.head()

,type,Total students,budget,reading_score,math_score,total math pass,total reading pass,% passing math,% passing reading
school,,,,,,,,,
Huang High School,District,2917,1910635,81.182722,76.629414,2533,2917,86.835790,100.0
Figueroa High School,District,2949,1884411,81.158020,76.711767,2549,2949,86.436080,100.0
Shelton High School,Charter,1761,1056600,83.725724,83.359455,1761,1761,100.000000,100.0
Hernandez High School,District,4635,3022020,80.934412,77.289752,4007,4635,86.450917,100.0
Griffin High School,Charter,1468,917500,83.816757,83.351499,1468,1468,100.000000,100.0


In [261]:
#add column for per student budget & overall passing rate
school_sumry_df["Per student budget"] = school_sumry_df["budget"]/school_sumry_df["Total students"]
school_sumry_df["Overall Passing Rate"] = (school_sumry_df["% passing math"] + school_sumry_df["% passing reading"])/2
school_sumry_df.head()

,type,Total students,budget,reading_score,math_score,total math pass,total reading pass,% passing math,% passing reading,Per student budget,Overall Passing Rate
school,,,,,,,,,,,
Huang High School,District,2917,1910635,81.182722,76.629414,2533,2917,86.835790,100.0,655.0,93.417895
Figueroa High School,District,2949,1884411,81.158020,76.711767,2549,2949,86.436080,100.0,639.0,93.218040
Shelton High School,Charter,1761,1056600,83.725724,83.359455,1761,1761,100.000000,100.0,600.0,100.000000
Hernandez High School,District,4635,3022020,80.934412,77.289752,4007,4635,86.450917,100.0,652.0,93.225458
Griffin High School,Charter,1468,917500,83.816757,83.351499,1468,1468,100.000000,100.0,625.0,100.000000


In [262]:
#some more renaming
school_sumry_df = school_sumry_df.rename(columns = {"type":"School type"})
school_sumry_df = school_sumry_df.rename(columns = {"budget":"total budget"})
school_sumry_df = school_sumry_df.rename(columns = {"math_score":"Avg math score"})
school_sumry_df = school_sumry_df.rename(columns = {"reading_score":"Avg reading score"})
school_sumry_df.head()

,School type,Total students,total budget,Avg reading score,Avg math score,total math pass,total reading pass,% passing math,% passing reading,Per student budget,Overall Passing Rate
school,,,,,,,,,,,
Huang High School,District,2917,1910635,81.182722,76.629414,2533,2917,86.835790,100.0,655.0,93.417895
Figueroa High School,District,2949,1884411,81.158020,76.711767,2549,2949,86.436080,100.0,639.0,93.218040
Shelton High School,Charter,1761,1056600,83.725724,83.359455,1761,1761,100.000000,100.0,600.0,100.000000
Hernandez High School,District,4635,3022020,80.934412,77.289752,4007,4635,86.450917,100.0,652.0,93.225458
Griffin High School,Charter,1468,917500,83.816757,83.351499,1468,1468,100.000000,100.0,625.0,100.000000


In [263]:
#sort school_sumry_df in descending order to get the schools with 5 highest passing rate
school_highest_df = school_sumry_df.sort_values("Overall Passing Rate",ascending=False)
school_highest_df.head()

,School type,Total students,total budget,Avg reading score,Avg math score,total math pass,total reading pass,% passing math,% passing reading,Per student budget,Overall Passing Rate
school,,,,,,,,,,,
Shelton High School,Charter,1761,1056600,83.725724,83.359455,1761,1761,100.0,100.0,600.0,100.0
Griffin High School,Charter,1468,917500,83.816757,83.351499,1468,1468,100.0,100.0,625.0,100.0
Wilson High School,Charter,2283,1319574,83.989488,83.274201,2283,2283,100.0,100.0,578.0,100.0
Cabrera High School,Charter,1858,1081356,83.975780,83.061895,1858,1858,100.0,100.0,582.0,100.0
Holden High School,Charter,427,248087,83.814988,83.803279,427,427,100.0,100.0,581.0,100.0


In [264]:
#sort school_sumry_df in descending order to get the schools with 5 highest passing rate
school_lowest_df = school_sumry_df.sort_values("Overall Passing Rate")
school_lowest_df.head()

,School type,Total students,total budget,Avg reading score,Avg math score,total math pass,total reading pass,% passing math,% passing reading,Per student budget,Overall Passing Rate
school,,,,,,,,,,,
Figueroa High School,District,2949,1884411,81.158020,76.711767,2549,2949,86.436080,100.0,639.0,93.218040
Rodriguez High School,District,3999,2547363,80.744686,76.842711,3457,3999,86.446612,100.0,637.0,93.223306
Hernandez High School,District,4635,3022020,80.934412,77.289752,4007,4635,86.450917,100.0,652.0,93.225458
Johnson High School,District,4761,3094650,80.966394,77.072464,4128,4761,86.704474,100.0,650.0,93.352237
Huang High School,District,2917,1910635,81.182722,76.629414,2533,2917,86.835790,100.0,655.0,93.417895


In [265]:
#Group student_df by school & grade to get avg math score per grade for each school
math_df = pd.DataFrame(student_df.groupby(["school","grade"])["math_score"].mean())
math_df.unstack()

math_score                                 
grade                       10th       11th       12th        9th
school                                                           
Bailey High School     76.996772  77.515588  76.492218  77.083676
Cabrera High School    83.154506  82.765560  83.277487  83.094697
Figueroa High School   76.539974  76.884344  77.151369  76.403037
Ford High School       77.672316  76.918058  76.179963  77.361345
Griffin High School    84.229064  83.842105  83.356164  82.044010
Hernandez High School  77.337408  77.136029  77.186567  77.438495
Holden High School     83.429825  85.000000  82.855422  83.787402
Huang High School      75.908735  76.446602  77.225641  77.027251
Johnson High School    76.691117  77.491653  76.863248  77.187857
Pena High School       83.372000  84.328125  84.121547  83.625455
Rodriguez High School  76.612500  76.395626  77.690748  76.859966
Shelton High School    82.917411  83.383495  83.778976  83.420755
Thomas High School     83.087886  83.498795  83.497041  83.590022
Wilson High School     83.724422  83.195326  83.035794  83.085578
Wright High School     84.010288  83.836782  83.644986  83.264706

In [266]:
#Group student_df by school & grade to get avg reading score per grade for each school
reading_df = pd.DataFrame(student_df.groupby(["school","grade"])["reading_score"].mean())
reading_df.unstack()

reading_score                                 
grade                          10th       11th       12th        9th
school                                                              
Bailey High School        80.907183  80.945643  80.912451  81.303155
Cabrera High School       84.253219  83.788382  84.287958  83.676136
Figueroa High School      81.408912  80.640339  81.384863  81.198598
Ford High School          81.262712  80.403642  80.662338  80.632653
Griffin High School       83.706897  84.288089  84.013699  83.369193
Hernandez High School     80.660147  81.396140  80.857143  80.866860
Holden High School        83.324561  83.815534  84.698795  83.677165
Huang High School         81.512386  81.417476  80.305983  81.290284
Johnson High School       80.773431  80.616027  81.227564  81.260714
Pena High School          83.612000  84.335938  84.591160  83.807273
Rodriguez High School     80.629808  80.864811  80.376426  80.993127
Shelton High School       83.441964  84.373786  82.781671  84.122642
Thomas High School        84.254157  83.585542  83.831361  83.728850
Wilson High School        84.021452  83.764608  84.317673  83.939778
Wright High School        83.812757  84.156322  84.073171  83.833333